<a href="https://colab.research.google.com/github/Wesley-Janson/transformers_for_human_vs_ai_text_identification/blob/main/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import relevant packages and data_loader.py
#import data_loader

# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
import matplotlib.pyplot as plt

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torchtext.data.utils import get_tokenizer
from sklearn.model_selection import train_test_split

In [3]:
def load_data(csv):
  # Reads the raw csv file and split into
  # sentences (x) and target (y)
  df = pd.read_csv(csv)
  
  
  text = df['intro'].values
  labels = df['type'].values
  return labels,text

In [4]:

# This function processes training data, establishing number IDs for each vocabulary word,
# converting word sequence into ID sequence (input_as_ids), and providing dict
# to map from word to its ID (word2id), and list to map from ID back to word (id2word)
def process_training_data(corpus_text):
        """Tokenizes a text file."""
        # Create the model's vocabulary and map to unique indices
        word2id = {}
        id2word = []
        list_of_inputs = []
        for entry in corpus_text:
            for i,word in enumerate(entry):
                if 7<i<=30:
                    if word not in word2id:
                        id2word.append(word)
                        word2id[word] = len(id2word) - 1

            # Convert string of text into string of IDs in a tensor for input to model
            input_as_ids = []
            for i,word in enumerate(entry):
                if 7<i<=30:
                    input_as_ids.append(word2id[word])
            if len(input_as_ids) == 23:
              list_of_inputs.append(input_as_ids)
            # final_ids = torch.LongTensor(input_as_ids)
        list_of_inputs = torch.Tensor(list_of_inputs)

        return list_of_inputs,word2id,id2word

In [6]:
from google.colab import files
 
 
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
labels,text = load_data('data.csv')
train_x, val_x, train_y, val_y = train_test_split(text, labels, test_size = 0.8)
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_x), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [9]:
# Run data loader
labels,text = load_data('data.csv')
train_x, val_x, train_y, val_y = train_test_split(text, labels, test_size = 0.8)
tokenizer = get_tokenizer("basic_english")
list_of_tokens_train = [tokenizer(x) for x in train_x]
list_of_tokens_val = [tokenizer(x) for x in val_x]

train_x,word2id,id2word = process_training_data(list_of_tokens_train)

val_x,word2id,id2word = process_training_data(list_of_tokens_val)

train_y = torch.Tensor(train_y)
val_y = torch.Tensor(val_y)
#############################
#### SPLIT INTO TRAIN/TEST HERE??????
#############################

In [37]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(text)
vectorizer.vocabulary_

{'Sexhow': 386088,
 'railway': 601844,
 'station': 620624,
 'was': 642261,
 'located': 564714,
 'in': 551147,
 'the': 628990,
 'town': 631646,
 'of': 582385,
 'on': 583088,
 'Cumbrian': 105197,
 'Coast': 96132,
 'Line': 251266,
 'North': 307104,
 'West': 463711,
 'England': 135283,
 'The': 424933,
 'opened': 583401,
 'by': 500728,
 'Lancashire': 243489,
 'and': 486225,
 'Yorkshire': 473922,
 'Railway': 353044,
 'October': 311564,
 '1870': 3895,
 'It': 204464,
 'closed': 508116,
 'to': 630725,
 'passengers': 588217,
 'January': 207926,
 '1950': 4678,
 'goods': 540860,
 '12': 1611,
 'May': 272566,
 '1965': 4753,
 'building': 499997,
 'is': 554792,
 'now': 580891,
 'private': 597225,
 'residence': 605537,
 'There': 425525,
 'small': 617079,
 'amount': 485705,
 'trackage': 631778,
 'remaining': 604690,
 'near': 578012,
 'used': 638200,
 'currently': 515077,
 'local': 564682,
 'agricultural': 483500,
 'business': 500529,
 'In': 199846,
 'Finnish': 148029,
 'folklore': 535178,
 'all': 484614

In [41]:
from keras.preprocessing.text import Tokenizer
train_sentences, val_sentences, train_labels, val_labels = train_test_split(text, labels, test_size = 0.8)
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_sentences)

X_train = tokenizer.texts_to_sequences(train_sentences)
X_test = tokenizer.texts_to_sequences(val_sentences)

vocab_size = len(tokenizer.word_index) + 1 

In [42]:
from keras.models import Sequential
from keras import layers
maxlen=23
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 23, 50)            13426400  
                                                                 
 flatten_1 (Flatten)         (None, 1150)              0         
                                                                 
 dense_4 (Dense)             (None, 10)                11510     
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 13,437,921
Trainable params: 13,437,921
Non-trainable params: 0
_________________________________________________________________


In [58]:
print(train_labels)
print(type(train_labels))
print(type(X_train))
X_train= np.array(X_train)
X_test = np.array(X_test)
print(X_train[0])


tf.Tensor([1 0 1 ... 0 1 1], shape=(60000,), dtype=int64)
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
[8, 19, 36, 450, 4, 237, 34, 1351, 3, 169, 543, 3, 1, 63, 840, 1452, 2076, 11, 29, 557, 10, 723, 3, 1, 169, 57, 1, 1, 411, 92, 62, 26, 21, 336, 3, 5, 103, 2, 111, 3384, 2095, 283, 47, 2032, 362, 1, 344, 4, 318, 26, 21, 22, 336, 12, 1595, 3, 405, 247, 7, 24, 3, 888, 329, 976, 6, 861, 1788, 5, 774, 4, 587, 19, 26, 21, 30, 968, 34, 1367, 91, 3, 1, 2910, 2, 104, 2811, 101, 29, 825, 77, 1351, 3, 169, 543, 3, 1, 63, 840, 29, 4069, 529, 61, 26, 7, 1325, 10, 723, 3, 1, 169, 57, 1, 1, 411, 92, 62, 26, 21, 336, 3, 5, 103, 2, 111, 3384, 2095, 283, 47, 2032, 362, 1, 344, 318, 4, 255, 228, 26, 8, 22, 42, 11, 29, 223, 10, 841, 4915, 2636, 12, 1, 3283, 2365, 3112, 556, 537, 96, 17, 344, 6, 181, 3, 226, 26, 731, 10, 912, 3, 1, 790, 3113, 309, 1105, 57, 1, 4, 32, 1, 273, 4, 1, 419, 11, 318, 17, 297, 6]


<ipython-input-58-3fd0bdea7cae>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(X_test)


In [66]:
import tensorflow as tf
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int64) 
print(X_train.shape)
X_train = tf.convert_to_tensor(X_train, dtype=tf.int64) 
val_labels = tf.convert_to_tensor(val_labels, dtype=tf.int64) 
X_test = tf.convert_to_tensor(X_test, dtype=tf.int64) 

(60000,)


ValueError: ignored

In [62]:
history = model.fit(X_train, train_labels,
                    epochs=20,
                    verbose=False,
                    validation_data=(X_test, val_labels),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, train_labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, val_labels, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

ValueError: ignored

In [12]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [16]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-04-30 21:29:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-30 21:29:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-30 21:29:34--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [17]:
!unzip glove*.zip


Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [18]:
!ls
!pwd

data.csv	   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
/content


In [19]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('glove.6B.50d.txt', tokenizer.word_index, embedding_dim)

In [20]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.5382524905365174

In [23]:
from keras.models import Sequential
from keras import layers
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=23, 
                           trainable=False))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 50)            13486050  
                                                                 
 global_max_pooling1d (Globa  (None, 50)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                510       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 13,486,571
Trainable params: 521
Non-trainable params: 13,486,050
_________________________________________________________________


In [35]:
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
X_train = np.array(X_train)
X_test = np.array(X_test)

<ipython-input-35-a7881001f13a>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(X_train)
<ipython-input-35-a7881001f13a>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(X_test)


In [36]:
history = model.fit(X_train, train_labels,
                    epochs=50,
                    verbose=False,
                    validation_data=(X_test, val_labels),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, train_labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, val_labels, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

ValueError: ignored

In [67]:
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [68]:
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size=[5000], 
                  embedding_dim=[50],
                  maxlen=[100])

In [71]:
!pip install keras.preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00


In [74]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from keras_preprocessing.sequence import pad_sequences
# Main settings
epochs = 20
embedding_dim = 50
maxlen = 100
output_file = 'data/output.txt'

# Run grid search for each source (yelp, amazon, imdb)
y,sentences = load_data('data.csv')

# Train-test split
sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences, y, test_size=0.25, random_state=1000)

# Tokenize words
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# Pad sequences with zeros
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Parameter grid for grid search
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size=[vocab_size],
                  embedding_dim=[embedding_dim],
                  maxlen=[maxlen])
model = KerasClassifier(build_fn=create_model,
                        epochs=epochs, batch_size=10,
                        verbose=False)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                          cv=4, verbose=1, n_iter=5)
grid_result = grid.fit(X_train, y_train)

# Evaluate testing set
test_accuracy = grid.score(X_test, y_test)

print(test_accuracy)

<ipython-input-74-3f8c5c287245>:36: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model,


Fitting 4 folds for each of 5 candidates, totalling 20 fits


KeyboardInterrupt: ignored

In [11]:
# Import relevant packages and data_loader.py
#import data_loader

# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
import matplotlib.pyplot as plt

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torchtext.data.utils import get_tokenizer
from sklearn.model_selection import train_test_split

In [12]:
from google.colab import files
 
 
uploaded = files.upload()

MessageError: ignored

In [4]:
def load_data(csv):
  # Reads the raw csv file and split into
  # sentences (x) and target (y)
  df = pd.read_csv(csv)
  
  
  text = df['intro'].values
  labels = df['type'].values
  return labels,text

In [9]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
labels,text = load_data('data.csv')
train_x, val_x, train_y, val_y = train_test_split(text, labels, test_size = 0.8)
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_x), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

FileNotFoundError: ignored

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [ ]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('data.csv')
df = df[['intro','type']]
training_dataset, test_dataset = train_test_split(df, train_size=0.75)

In [ ]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)
dataloader = DataLoader(training_dataset, batch_size=8, shuffle=False, collate_fn=collate_batch)
data = dataloader.dataset

print(data)


In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [6]:
print(training_dataset)

NameError: ignored

In [7]:
num_class = 2
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

NameError: ignored

In [8]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [2]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_dataset = to_map_style_dataset(training_dataset)
test_dataset = to_map_style_dataset(test_dataset)
print(train_dataset)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])
print(split_train_)
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

NameError: ignored

In [ ]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))